<a href="https://colab.research.google.com/github/ejyepezm/PPIA/blob/main/unidad_3_aplicaciones_python/2_MLflow_Tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 💊 Cápsula 2: El Detective de Experimentos (MLflow)
**Tema:** Tracking de Experimentos, Registro de Métricas y Comparación de Modelos.

## 1. El Caos de los Hiperparámetros

En ML, nunca entrenas un solo modelo. Pruebas cientos:
*   "RandomForest con 10 árboles"
*   "RandomForest con 50 árboles"
*   "Con escalado vs sin escalado"

Si anotas esto en un Excel o confías en tu memoria, perderás el mejor modelo.

### La Solución: MLflow
MLflow es una herramienta (Open Source) que actúa como un "Diario de Laboratorio" automático.
Dentro de un bloque `with mlflow.start_run():`, todo lo que hagas queda guardado para siempre:
*   `log_param()`: Qué configuración usaste (ej. n_estimators=100).
*   `log_metric()`: Qué resultado obtuviste (ej. accuracy=0.95).
*   `log_model()`: Guarda el archivo del modelo (.pkl) automáticamente.

In [ ]:
# Instalamos MLflow (silencioso para no ensuciar la salida)
!pip install mlflow -q

import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Preparamos datos (Iris)
data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2)

# Configuración básica (Local)
mlflow.set_experiment("Experimento_Iris_Clase")

In [ ]:
# --- DEMOSTRACIÓN: Un solo experimento ---

print("Iniciando entrenamiento...")

# Iniciamos el "Run" (La ejecución)
with mlflow.start_run(run_name="Modelo_Bosque_Pequeño"):

    # 1. Definimos parámetros
    n_arboles = 10
    profundidad = 3

    # 2. Registramos parámetros (Para no olvidarlos)
    mlflow.log_param("n_estimators", n_arboles)
    mlflow.log_param("max_depth", profundidad)

    # 3. Entrenamos
    clf = RandomForestClassifier(n_estimators=n_arboles, max_depth=profundidad)
    clf.fit(X_train, y_train)

    # 4. Evaluamos
    predicciones = clf.predict(X_test)
    acc = accuracy_score(y_test, predicciones)

    # 5. Registramos la métrica (El resultado)
    print(f"Accuracy obtenido: {acc}")
    mlflow.log_metric("accuracy", acc)

    # 6. Guardamos el modelo (Artefacto)
    mlflow.sklearn.log_model(clf, "modelo_random_forest")

print("✅ Experimento registrado en MLflow.")

## 🔥 Micro-Desafío: El Bucle de Optimización

Queremos encontrar el número óptimo de árboles para nuestro Bosque Aleatorio.
Probar manualmente es aburrido.

**Tu Misión:**
1.  Crea una lista de configuraciones: `lista_arboles = [10, 50, 100, 200]`.
2.  Haz un bucle `for` que recorra esa lista.
3.  Dentro del bucle, inicia un `mlflow.start_run()`.
4.  Entrena el modelo con ese número de árboles, registra el parámetro y la métrica de accuracy.
5.  **Bonus:** Al final, usa `mlflow.search_runs()` para mostrar un DataFrame con todos los resultados y ver cuál ganó.

In [ ]:
# Lista de hiperparámetros a probar
lista_arboles = [5, 20, 50, 100]

print("🧪 Iniciando búsqueda de hiperparámetros...")

for n in lista_arboles:
    # --- TU CÓDIGO AQUÍ ---
    # TODO: Inicia el run con un nombre dinámico ej: f"Run_{n}_arboles"
    with mlflow.start_run(run_name=f"Run_{n}_arboles"):

        # TODO: Loguea el parámetro n_estimators

        # Entrenamiento (Te lo doy hecho)
        model = RandomForestClassifier(n_estimators=n, random_state=42)
        model.fit(X_train, y_train)
        acc = accuracy_score(y_test, model.predict(X_test))

        # TODO: Loguea la métrica accuracy

        print(f"--> Arboles: {n} | Accuracy: {acc:.4f}")

# --- VALIDACIÓN Y ANÁLISIS ---
print("\n📊 Resumen de Experimentos (Pandas DataFrame):")
# MLflow nos permite bajar los resultados como tabla
resultados = mlflow.search_runs()
print(resultados[['params.n_estimators', 'metrics.accuracy']].sort_values(by='metrics.accuracy', ascending=False))